In [ ]:
from sec_edgar_downloader import Downloader
import os
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'AMD', 
           'BA', 'V', 'SPY', 'SPCE', 'BABA', 'MSTR', 'DIS', 'PYPL', 'SHOP', 'COIN', 
            'INTC', 'CSCO', 'IBM', 'GE', 'WMT', 'T']

In [43]:
dl = Downloader("NYU", "tan4742@nyu.edu", "../data/sec_data")

In [50]:
def extract_financials(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        
        # Extract relevant metadata information from the document
        filing_data = {}

        # Helper function to safely find the next tag for metadata
        def safe_find(text):
            tag = soup.find(text=text)
            if tag:
                return tag.find_next().text.strip() if tag.find_next() else None
            return None
        
        # Extract metadata
        filing_data["accession_number"] = safe_find("ACCESSION NUMBER:")
        filing_data["conformed_submission_type"] = safe_find("CONFORMED SUBMISSION TYPE:")
        filing_data["company_name"] = safe_find("COMPANY CONFORMED NAME:")
        filing_data["fiscal_year_end"] = safe_find("FISCAL YEAR END:")
        filing_data["date_of_filing"] = safe_find("FILED AS OF DATE:")
        
        # Extracting document metadata (if available)
        filing_data["form_type"] = safe_find("FORM TYPE:")
        filing_data["sec_act"] = safe_find("SEC ACT:")
        filing_data["sec_file_number"] = safe_find("SEC FILE NUMBER:")
        filing_data["film_number"] = safe_find("FILM NUMBER:")
        
        # Extracting business and mailing address (if available)
        filing_data["business_address"] = safe_find("BUSINESS ADDRESS:")
        filing_data["mail_address"] = safe_find("MAIL ADDRESS:")
        
        # Extracting HTML content from <DOCUMENT> for financials
        document_tag = soup.find("DOCUMENT")
        if document_tag:
            filing_data["document_html"] = document_tag.text.strip()
        
        return filing_data

In [51]:
def process_tickers(tickers, base_path):
    all_data = []
    
    for ticker in tickers:
        # Set the base directory for this ticker
        ticker_dir = os.path.join(base_path, ticker, "8-K")
        
        # Iterate through each submission folder for the ticker
        for subdir, _, files in os.walk(ticker_dir):
            for file in files:
                if file == "full-submission.txt":
                    file_path = os.path.join(subdir, file)
                    
                    # Extract the relevant data from the full submission file
                    filing_data = extract_financials(file_path)
                    
                    # Add extracted data to the list
                    all_data.append(filing_data)
    
    # Convert the list of dictionaries into a pandas DataFrame
    df = pd.DataFrame(all_data)
    
    # Save the DataFrame to a CSV or JSON file
    df.to_csv("data/cleaned_tables/filing_data.csv", index=False)

In [52]:
base_path = "../data/sec_data/sec-edgar-filings"

In [53]:
process_tickers(tickers, base_path)

/var/folders/zc/mk397vzd3rndczvtnq2xwwzr0000gn/T/ipykernel_3426/4192151999.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = soup.find(text=text)


ParserRejectedMarkup: The markup you provided was rejected by the parser. Trying a different parser or a different encoding may help.

Original exception(s) from parser:
 AssertionError: expected name token at '<![(V#1(2\\Y32.RXI/Y('